In [ ]:
!pip install transformers -U

In [3]:
!pip install refractml

     |████████████████████████████████| 51kB 6.8MB/s eta 0:00:011
Processing /home/mosaic-ai/.cache/pip/wheels/ab/d0/0e/613976a1b51b5654859e2a82ade64329859bce431e280f2a39/shutils-0.1.0-cp38-none-any.whl
     |████████████████████████████████| 61kB 17.3MB/s eta 0:00:01
     |████████████████████████████████| 143kB 14.6MB/s eta 0:00:01
     |████████████████████████████████| 71kB 27.8MB/s eta 0:00:01
     |████████████████████████████████| 706kB 17.1MB/s eta 0:00:01
     |████████████████████████████████| 51kB 67.5MB/s eta 0:00:01
     |████████████████████████████████| 71kB 43.5MB/s eta 0:00:01
     |████████████████████████████████| 9.8MB 23.2MB/s eta 0:00:01
     |████████████████████████████████| 143kB 105.1MB/s eta 0:00:01
     |████████████████████████████████| 163kB 123.8MB/s eta 0:00:01
     |████████████████████████████████| 71kB 103.4MB/s ta 0:00:01
     |████████████████████████████████| 34.5MB 117.2MB/s eta 0:00:01
     |████████████████████████████████| 307kB 87.1MB/s eta 0:

In [4]:
!pip uninstall urllib3 -y

Uninstalling urllib3-2.1.0:
  Successfully uninstalled urllib3-2.1.0


In [5]:
!pip install urllib3==1.26.15

     |████████████████████████████████| 143kB 5.7MB/s eta 0:00:01
ERROR: refractml 1.0.3 has requirement PyYAML==6.0, but you'll have pyyaml 6.0.1 which is incompatible.
ERROR: torchvision 0.15.2+cpu has requirement torch==2.0.1, but you'll have torch 2.1.1 which is incompatible.
ERROR: jupyterlab-server 2.23.0 has requirement jsonschema>=4.17.3, but you'll have jsonschema 3.2.0 which is incompatible.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

In [ ]:
from refractml import *
from refractml.constants import MLModelFlavours

# # new score functions
from mosaic_utils.ai.score.base import ScoreBase
from typing import Tuple, Union, List, Any
import numpy as np

In [ ]:
######################################################################################################
# Model Registration

from refractml import *
from refractml.constants import MLModelFlavours

# new score functions
from mosaic_utils.ai.score.base import ScoreBase
from typing import Tuple, Union, List, Any
class ScoreTemplateExample(ScoreBase):
    """
    This Class Demonstrate How To Implements ScoreBase Interface Class And It Basic Usage.
    """    
    def __init__(self):
        super().__init__()
        import os
        os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
        from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
        import torch
        from torch import cuda
        self.model_loaded = None
        self.tokenizer = None
#        self.token_path = '/data/models/mistralai_base/token.pkl'
#        self.model_path = '/data/models/mistralai_base/qa.pkl'
        if self.model_loaded is None:
            print("Load Model From Data Volume")
            self.model_loaded = AutoModelForCausalLM.from_pretrained("/data/huggingface/cache/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f",device_map = 'cpu',torch_dtype=torch.bfloat16)
#             self.model_loaded =torch.load(self.model_path)

        if self.tokenizer is None:
            print("tokenizer object is loading from data section")
            self.tokenizer = AutoTokenizer.from_pretrained("/data/huggingface/cache/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f")
#             self.tokenizer = torch.load(self.token_path)
        
        print("model and tokens are loaded")
    def request_processing_fn(self, request) :
        """
        Processes Request Object -> List[Input data, ..]. It could be:
               A List Mapping of All Value Can Be one of : 
                   - List[ [Feature_Value1, Feature_Value2, ...], [...] ]
                   - List[numpy.array(), numpy.array(), ...]
                   - List[tf.Tensor, tf.Tensor, tf.Tensor, ...]
                   - List[ SingleSample, SingleSample]
                   
        :return: (n_inputs, payload's)
        
        Warnings:
        1. Do not reshape your final output for single sample here, do it in prediction.
           Else payloads will be invalidated for extraction at raw and extraction level.
        """
        final_payload = []
        raw_payload = request.json["payload"]
        return (1, raw_payload) 
    
    def pre_processing_fn(self,payload):
        # All preprocessing step must occur in this section
        # Takes Single Sample -> Returns Single Sample
        
        # Not Doing Any Preprocessing Hence Returned payload
        print("payload is ", payload)
        
        return payload

    def prediction_fn(self,
                      model: Any,
                      pre_processed_input 
                      ):
        """
                Does the main prediction on pre_processed_input(Single Sample) using supplied model .

                :param model: Supported Model
                :param pre_processed_input: Single Preprocessed Payload
                :return: Prediction Value From the model
                
                Important Notes:
                - Reshape your data array.reshape(1, -1) before predictions as it contains a single sample.
                    
        """
#         device = 'cuda' if cuda.is_available() else 'cpu'
#         print(f'Device: ------------- {device}')
        input_data = pre_processed_input
        print(type(input_data))
        print(input_data)
        prompt = input_data
        pipe = pipeline(
        "text-generation", 
        model= self.model_loaded, 
        tokenizer = self.tokenizer, 
        torch_dtype=torch.bfloat16
    )
        print("pipe object is loaded")
        sequences = pipe(
            prompt,
            do_sample=True,
            max_new_tokens=100, 
            temperature=0.7, 
            top_k=50, 
            top_p=0.95,
            num_return_sequences=1,
        )
        print(sequences[0]['generated_text'])
        print("inference generated")
        return sequences[0]['generated_text']

    class Meta:    
        # List of Callables() can be attached For Calling After AnSd Before Scoring
        def __init__(self):
            self.name = "Pre Hooked Me !"
            self.pre_call_hooks.append(self.print_)
        def print_(self):
            print(self.name)
        pre_call_hooks = []
        post_call_hooks = []

In [ ]:
prompt = "How to reverse a string in python?"
import requests
req = requests.Request()
req.json = {"payload":prompt
            }

In [ ]:
score_ = ScoreTemplateExample()

In [ ]:
model_predictions = score_.score(None, req, dry_run=True)

In [ ]:
register_model(None,
               ScoreTemplateExample,
               "Starchat_Beta_Second_Version",
               "Starchat_Beta_for_code_Generation",
               MLModelFlavours.pytorch,
               init_script="pip install SentencePiece \\n pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu"
              )